## Final Project Submission

Please fill out:
* Student name: Abdulrahman Aber, Benjamin Bai, Charles Pan, Kevin Rivera
* Student pace: Full Time
* Scheduled project review date/time: 
* Instructor name: David Elliott
* Blog post URL:


# Imports, Data Frame Initializations

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sq
import seaborn as sns
conn = sq.connect('./zippedData/im.db/im.db')
cur = conn.cursor()

In [9]:
movie_basics_df = pd.read_sql('''select * from movie_basics''', conn)

In [13]:
movie_ratings_df = pd.read_sql("""select * from movie_ratings""",conn)

In [14]:
persons_df = pd.read_sql("""select * from persons""",conn)

In [15]:
principals_df = pd.read_sql("""select * from principals""",conn)

In [19]:
tmdb_movies_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz')

In [20]:
tn_movie_budgets_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz',
                                 parse_dates=['release_date'])

In [33]:
tn_movie_budgets_df.info()

# This dataframe will be used to derive net_return and ROI as a percentage, specifically
# production_budget and worldwide_gross.

# We believe net_return and ROI% are the key metrics for an executive team weighing different
# investment options. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1851 entries, 1 to 5000
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1851 non-null   int64         
 1   release_date       1851 non-null   datetime64[ns]
 2   movie              1851 non-null   object        
 3   production_budget  1851 non-null   float64       
 4   domestic_gross     1851 non-null   float64       
 5   worldwide_gross    1851 non-null   float64       
 6   release_year       1851 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 115.7+ KB


# TN Movie Budgets Clean Up Process

In [22]:
# Remove $ from production_budget, convert to float
tn_movie_budgets_df['production_budget'] = tn_movie_budgets_df['production_budget'].str.replace(',', '').str.replace('$', '').astype(float)

# Remove $ from domestic_gross, convert to float
tn_movie_budgets_df['domestic_gross'] = tn_movie_budgets_df['domestic_gross'].str.replace(',', '').str.replace('$', '').astype(float)

# Remove $ from worldwide_gross, convert to float
tn_movie_budgets_df['worldwide_gross'] = tn_movie_budgets_df['worldwide_gross'].str.replace(',', '').str.replace('$', '').astype(float)

# new column release_year from release_date to filter based on time frame in scope
tn_movie_budgets_df['release_year'] = pd.DatetimeIndex(tn_movie_budgets_df['release_date']).year

# time frame: 2010-Present; SAG definition of theatrical budget: > 2 million
tn_movie_budgets_df = tn_movie_budgets_df[(tn_movie_budgets_df.release_year >= 2010) & (tn_movie_budgets_df.production_budget > 1999999)]

# Merge TN Movie Budgets with IMDB Movie Basics for Genres Analysis

In [23]:
# rename movie_basics column to match tn_movie_budgets
movie_basics_df.rename(columns={"primary_title":"movie"}, inplace = True)

# merge movie_budgets and imdb movie_basics
moviebudgets_moviebasics_df = tn_movie_budgets_df.merge(movie_basics_df, how='inner', on='movie')

# drop columns that are not relevant to Genre analysis
moviebudgets_moviebasics_df.drop(columns = ['id','release_date', 'movie_id', 'original_title', 'runtime_minutes', 'domestic_gross' ],axis=1,inplace=True)

# remove duplicate movies
moviebudgets_moviebasics_df.drop_duplicates(subset='movie', keep='first', inplace=True, ignore_index=False)

# add column for net_return
moviebudgets_moviebasics_df['net_return'] = (moviebudgets_moviebasics_df['worldwide_gross'] - moviebudgets_moviebasics_df['production_budget'])

# add column for ROI as a %
moviebudgets_moviebasics_df['roi_percent'] = ((moviebudgets_moviebasics_df['net_return'] / moviebudgets_moviebasics_df['production_budget'])*100)